In [1]:
from model import *

In [2]:
timesteps=10
dt = 1/247
simulations_num=SIM_NUM
r_risks, r_instruments, act_risks, act_instruments = get_data()
logger.info('Data is fetched')

stoch_gen = stoch_wrapper(r_risks)
models = train_models(r_risks, r_instruments) #мы ограничиваем трейн?

calculated_risks=dict()
for it in range(max(RISKS_LOOKBACK, INSTRUMENTS_LOOKBACK,1), r_risks.shape[0]-timesteps):
    if it%100==0:
        logger.info(f'iteration {it} is in')
    local_pivot = max(RISKS_LOOKBACK, INSTRUMENTS_LOOKBACK)
    time = r_risks.index[it]
    risk_data = r_risks.iloc[       it-local_pivot:it+timesteps].values
    instr_data = r_instruments.iloc[it-local_pivot:it+timesteps]#what for?
    init_ins = act_instruments.iloc[  INSTRUMENTS_LOOKBACK+it,:].values.reshape(-1)
    instr_results =        np.zeros(shape = ( timesteps, instr_data.shape[1], simulations_num)) #Сюда запишем результат предсказаний по инструментам
    risk_factors_history = np.zeros(shape = ( timesteps, RISK_FACTORS_NUM,    simulations_num)) #Сюда запишем историю симуляции рисков

    #estimation of params for risk sim
    history_to_estimate_params = r_risks.iloc[local_pivot-RISKS_LOOKBACK:-timesteps]
    init_array = r_risks.iloc[-timesteps]

    gbm_params = params_for_gbm(history_to_estimate_params)
    for sim_id in range(simulations_num):
        stochs = stoch_gen(timesteps)

        sim = generate_gbm_sim(init_array, gbm_params, stochs, dt, timesteps)
        risk_factors_history[:,:,sim_id] = sim


    instruments_names = instr_data.columns

    broadcasted_lookback_history = np.stack([risk_data[:INSTRUMENTS_LOOKBACK] for _ in range(simulations_num)], axis=-1)
    # broadcasted_lookback_history = np.broadcast_to(risk_data[:INSTRUMENTS_LOOKBACK],(INSTRUMENTS_LOOKBACK,RISK_FACTORS_NUM,simulations_num))
    appended_history = np.concatenate((broadcasted_lookback_history,risk_factors_history),axis=0)


    simulated_instruments = np.stack([inference_models(appended_history[:,:,sim_id], models)for sim_id in range(simulations_num)], axis=-1)#array of shape (tsteps,instruments_num,simulations_num, )


    risks = calculate_risks(simulated_instruments)
    calculated_risks[time] = risks


var1={k:v[0] for k,v in calculated_risks.items()}
es1={k:v[1] for k,v in calculated_risks.items()}
var10={k:v[2] for k,v in calculated_risks.items()}
es10={k:v[3] for k,v in calculated_risks.items()}


2020-02-19 18:46:43,907 - base - INFO - Data is fetched
2020-02-19 18:46:43,914 - base - INFO - Decomposition calculated
2020-02-19 18:46:43,915 - base - INFO - Model training started
2020-02-19 18:46:44,262 - base - INFO - Model training is done
D:\Code\HSE-study\Риск-менеджмент\Проект 1\risk_project1\model.py:262: RuntimeWarning: Mean of empty slice.
  ESes = [prices[i][prices[i]<np.percentile(prices[i], q=2.5)].mean() for i in range(prices.shape[0]) ]
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Code\HSE-study\Риск-менеджмент\Проект 1\risk_project1\model.py:262: RuntimeWarning: Mean of empty slice.
  ESes = [prices[i][prices[i]<np.percentile(prices[i], q=2.5)].mean() for i in range(prices.shape[0]) ]
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rco

D:\Code\HSE-study\Риск-менеджмент\Проект 1\risk_project1\model.py:262: RuntimeWarning: Mean of empty slice.
  ESes = [prices[i][prices[i]<np.percentile(prices[i], q=2.5)].mean() for i in range(prices.shape[0]) ]
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
D:\Code\HSE-study\Риск-менеджмент\Проект 1\risk_project1\model.py:262: RuntimeWarning: Mean of empty slice.
  ESes = [prices[i][prices[i]<np.percentile(prices[i], q=2.5)].mean() for i in range(prices.shape[0]) ]
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 